In [1]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.sql.functions import col, udf
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, Row

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1574809323548_0007,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
rdd1 = sc.textFile("s3://akshaycl201/train.txt")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
rdd1_1=rdd1.map(lambda x: x.replace("__label__2","2"))
rdd1_2=rdd1_1.map(lambda x: x.replace("__label__1","1"))
rdd2 = rdd1_2.map(lambda x: x.split(" ",1))
rdd2.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[['2', 'Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^'], ['2', "The best soundtrack ever to anything.: I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny."], ['2', 'Amazing!: This soundtrack is my fav

In [4]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import rand 
from time import time
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, Row

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
rdd3 = rdd2.map(lambda x: Row(label=x[0],reviewText=x[1]))
rdd3.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(label='2', reviewText='Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^'), Row(label='2', reviewText="The best soundtrack ever to anything.: I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny."), Ro

In [6]:
# Converting an RDD to DataFrame
df=sqlContext.createDataFrame(rdd3)
# Registers this DataFrame as a temporary table using the given name
df.registerTempTable("df")
# convert String labels to Double type
df = df.withColumn("label", df.label)
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+
|label|          reviewText|
+-----+--------------------+
|    2|Stuning even for ...|
|    2|The best soundtra...|
|    2|Amazing!: This so...|
|    2|Excellent Soundtr...|
|    2|Remember, Pull Yo...|
|    2|an absolute maste...|
|    1|Buyer beware: Thi...|
|    2|Glorious story: I...|
|    2|A FIVE STAR BOOK:...|
|    2|Whispers of the W...|
|    1|The Worst!: A com...|
|    2|Great book: This ...|
|    2|Great Read: I tho...|
|    1|Oh please: I gues...|
|    1|Awful beyond beli...|
|    1|Don't try to fool...|
|    2|A romantic zen ba...|
|    2|Fashionable Compr...|
|    2|Jobst UltraSheer ...|
|    1|sizes recomended ...|
+-----+--------------------+
only showing top 20 rows

In [7]:
sqlContext.registerDataFrameAsTable(df, "table1")
df2 = sqlContext.sql("SELECT reviewText, label from table1 WHERE reviewText != ''")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df2.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|          reviewText|label|
+--------------------+-----+
|Stuning even for ...|    2|
|The best soundtra...|    2|
|Amazing!: This so...|    2|
|Excellent Soundtr...|    2|
|Remember, Pull Yo...|    2|
+--------------------+-----+
only showing top 5 rows

In [9]:
regexTokenizer = RegexTokenizer(inputCol="reviewText", outputCol="words", pattern="\\W")
# alternatively, pattern="\\w+", gaps(False)
countTokens = udf(lambda words: len(words), IntegerType())


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
regexTokenized = regexTokenizer.transform(df2)
regexTokenized.select("reviewText", "words") \
    .withColumn("tokens", countTokens(col("words"))).show(truncate=False)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [11]:
regexTokenized.show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+--------------------+
|          reviewText|label|               words|
+--------------------+-----+--------------------+
|Stuning even for ...|    2|[stuning, even, f...|
|The best soundtra...|    2|[the, best, sound...|
|Amazing!: This so...|    2|[amazing, this, s...|
|Excellent Soundtr...|    2|[excellent, sound...|
|Remember, Pull Yo...|    2|[remember, pull, ...|
|an absolute maste...|    2|[an, absolute, ma...|
|Buyer beware: Thi...|    1|[buyer, beware, t...|
|Glorious story: I...|    2|[glorious, story,...|
|A FIVE STAR BOOK:...|    2|[a, five, star, b...|
|Whispers of the W...|    2|[whispers, of, th...|
|The Worst!: A com...|    1|[the, worst, a, c...|
|Great book: This ...|    2|[great, book, thi...|
|Great Read: I tho...|    2|[great, read, i, ...|
|Oh please: I gues...|    1|[oh, please, i, g...|
|Awful beyond beli...|    1|[awful, beyond, b...|
|Don't try to fool...|    1|[don, t, try, to,...|
|A romantic zen ba...|    2|[a, romantic, zen...|


In [12]:
htf = HashingTF(inputCol="words", outputCol="features")
tf = htf.transform(regexTokenized)
train = tf[["label","features"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:

types = [f.dataType for f in df2.schema.fields]
types

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[StringType, StringType]

In [14]:

train2 = train.withColumn("label",train["label"].cast(DoubleType()))
types = [f.dataType for f in train2.schema.fields]
types

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[DoubleType, VectorUDT]

In [15]:

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
pipeline = Pipeline(stages=[lr])
model = pipeline.fit(train2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
rdd1_test = sc.textFile("s3://akshaycl201/test.txt")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
rdd1_1_test=rdd1_test.map(lambda x: x.replace("__label__2","2"))
rdd1_2_test=rdd1_1_test.map(lambda x: x.replace("__label__1","1"))
rdd2_test = rdd1_2_test.map(lambda x: x.split(" ",1))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
rdd3_test = rdd2_test.map(lambda x: Row(label=x[0],reviewText=x[1]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df_test=sqlContext.createDataFrame(rdd3_test)
# Registers this DataFrame as a temporary table using the given name
df_test.registerTempTable("df_test")

df_test = df_test.withColumn("label", df_test.label)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
sqlContext.registerDataFrameAsTable(df_test, "table2")
df2_test = sqlContext.sql("SELECT reviewText, label from table2 WHERE reviewText != ''")

df2_test.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|          reviewText|label|
+--------------------+-----+
|Great CD: My love...|    2|
|One of the best g...|    2|
|Batteries died wi...|    1|
|works fine, but M...|    2|
|Great for the non...|    2|
+--------------------+-----+
only showing top 5 rows

In [21]:
regexTokenized_test= regexTokenizer.transform(df2_test)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
regexTokenized_test.select("reviewText", "words") \
    .withColumn("tokens", countTokens(col("words"))).show(truncate=False)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------

In [23]:
tf_test = htf.transform(regexTokenized_test)
testSet = tf_test[["label","features"]]
testSet = testSet.withColumn("label",testSet["label"].cast(DoubleType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:

prediction = model.transform(testSet)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(prediction)
print("Test Error = %g " % (accuracy))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Test Error = 0.5